Step 1 - Data Import

Welcome in the workflow of analyzing growth data for Y. lipolytica Transcription Factors strains :)

The script is developed so that you can skip most of the parts and input as little as possible by yourself.
The code parts are all described with a # sign at the beginning of each python code cell - you don't need to change anything in them, just run them.
The parts of the code that needs Your involvement are subseeded with a markdown cell (such as this one) - which appear as a plain text. In each of such markdown cells Your task in the subsequent code will be explained.

Have fun watching Your data being analyzed ;)

In [10]:
# Perform all the necessary imports of libraries

from datetime import datetime
import glob
import pandas as pd
import numpy as np
import os


In this file you are only asked to insert the dilution factor for your OD600 values.

It is valid only if you used the same dilution factor throughout the whole experiment (with all conditions and timepoints). However if you diluted the cells differently through the course of your experiments, you have to enter a value of 1 in the below cell, and calculate the actual growth in your input files.

Meaning:
- if the dilutions are consistent you can input raw absorbance data, 
- if the dilutions change you have to calculate the growth yourself and use it as input files, and enter 1 as the dilution factor.

In [11]:
dilution_factor = float(os.environ.get("DILUTION_FACTOR", 1.0))  # 1.0 is the default
print(f"Using dilution factor: {dilution_factor}")

Using dilution factor: 1.0


In [12]:
# Assign ECY numbers used in the experiments to a list of names
# To change anything in the following three cells you have to keep the order the same 
# in ECY numbers, corresponding TF name and type of modification

numbers_ECY = '''
352
353
354
642
643
644
645
646
647
648
649
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
995
996
997
998
999
1000
1001
1002
1005
1006
1007
1016
1017
1018
1039
1040
1041
1042
1043
1044
1045
1049
1050
1051
F6
A3
A5
C6
G5
H5
A11
B10
B12
'''
list_ECY = [f"{item.strip()}" for item in numbers_ECY.strip().split('\n')]

In [13]:
# Assign TFs names used in the experiments to a list of names

names_TFs = '''
control
control
control
TF011
TF011
TF011
Mhy1
Mhy1
Mhy1
Hap1
Hap1
TF036
TF036
TF036
Dal81
Dal81
Dal81
Yas1
Yas1
Yas1
Msn4w
Msn4w
Msn4w
Msn4m
Msn4m
Msn4m
Msn4b
Msn4b
Msn4b
Hap1
Hap1
Dal81
Dal81
Dal81
Yas1
Yas1
Yas1
Msn4
Msn4
Msn4
TF009
TF009
TF009
Mhy1
Mhy1
Mhy1
TF011
TF011
TF011
TF009
TF036
TF036
TF036
control_prot
TF009
TF011
TF036
Yas1
Mhy1
Msn4
Dal81
Hap1
'''

list_TFs = [f"{item.strip()}" for item in names_TFs.strip().split('\n')]

In [14]:
# Assign modifications names used in the experiments to a list of names

names_Modifications = '''
control
control
control
OE
OE
OE
OE
OE
OE
OE
OE
OE
OE
OE
OE
OE
OE
OE
OE
OE
OE
OE
OE
OE
OE
OE
OE
OE
OE
KO
KO
KO
KO
KO
KO
KO
KO
KO
KO
KO
OE
OE
OE
KO
KO
KO
KO
KO
KO
KO
KO
KO
KO
OE_prot
OE_prot
OE_prot
OE_prot
OE_prot
OE_prot
OE_prot
OE_prot
OE_prot
'''

list_Modifications = [f"{item.strip()}" for item in names_Modifications.strip().split('\n')]

In [15]:
# Create a dictionary for each ECY number to the corresponding name of TF and type of modification

dictionaryTFs = dict(zip(list_ECY, list_TFs))
dictionaryModifications = dict(zip(list_ECY, list_Modifications))

In [16]:
DATA_PATH = os.path.join(os.getcwd(), "data")

results = [f for f in glob.glob(os.path.join(DATA_PATH, "results*.xlsx")) if not ":" in f]
print("Found files:", results)
if not results:
    print("No Excel files found in /app/data. Please upload files and re-run.")
    
strain_layout_upload = glob.glob(os.path.join(DATA_PATH, '*strain_layout.xlsx'))
if not strain_layout_upload:
    print('No strain layout template found in /app/data. Please upload file and re-run.')
print(strain_layout_upload)

Found files: ['\\\\wsl.localhost\\Ubuntu\\home\\marysia\\stress_resistance_msc\\data\\results_test_2.xlsx', '\\\\wsl.localhost\\Ubuntu\\home\\marysia\\stress_resistance_msc\\data\\results_test2_2.xlsx', '\\\\wsl.localhost\\Ubuntu\\home\\marysia\\stress_resistance_msc\\data\\results_test2_1.xlsx', '\\\\wsl.localhost\\Ubuntu\\home\\marysia\\stress_resistance_msc\\data\\results_test3_1.xlsx', '\\\\wsl.localhost\\Ubuntu\\home\\marysia\\stress_resistance_msc\\data\\results_test3_3.xlsx', '\\\\wsl.localhost\\Ubuntu\\home\\marysia\\stress_resistance_msc\\data\\results_test_3.xlsx', '\\\\wsl.localhost\\Ubuntu\\home\\marysia\\stress_resistance_msc\\data\\results_test_1.xlsx', '\\\\wsl.localhost\\Ubuntu\\home\\marysia\\stress_resistance_msc\\data\\results_test3_2.xlsx', '\\\\wsl.localhost\\Ubuntu\\home\\marysia\\stress_resistance_msc\\data\\results_test2_3.xlsx']
['\\\\wsl.localhost\\Ubuntu\\home\\marysia\\stress_resistance_msc\\data\\test_strain_layout.xlsx']


In [18]:
list_of_data = []

for file in results:
    
    # Load and process condition and time data from an excel filename
    filename = os.path.splitext(os.path.basename(file))[0]
    parts = filename.split('_')

    if len(parts) >= 3:
        condition = parts[1]
        time = parts[2]
    else:
        condition = 'Unknown'
        time = 'Unknown'
        
    # Load and process strain layout data
    strain_layout = pd.read_excel(strain_layout_upload[0], header=None)
    strain_layout = pd.melt(strain_layout, id_vars=None, value_name='ECY_number').dropna()

    # Load and process OD data
    od_results = pd.read_excel(file, header=None)  # use the current file from loop
    od_results = pd.melt(od_results, id_vars=None, value_name='absorbance_600').dropna()

    # Combine into one DataFrame
    data = pd.DataFrame({
        'ECY_number': strain_layout['ECY_number'],
        'absorbance_600': od_results['absorbance_600']
    })

    # Clean up and annotate
    data['ECY_number'] = data['ECY_number'].astype(str).str.split('.').str[0]
    data['absorbance_600'] = data['absorbance_600'].astype(float)
    data['TF'] = data['ECY_number'].map(dictionaryTFs).fillna('Unknown')
    data['modification'] = data['ECY_number'].map(dictionaryModifications).fillna('Unknown')
    data['growth'] = data['absorbance_600'] * dilution_factor
    data['condition'] = condition
    data['time'] = time
    data['time'] = data['time'].astype(int)
    data['strain_name'] = data['TF'] + '_' + data['modification']
    data['variant_column'] = data['strain_name'] + '_' + data['condition'] + '_' + data['time'].astype(str)
    
    # Append current dataframe to a list of data
    list_of_data.append(data)
    
# Update a dataframe with all results from experiments performed to date
all_data = pd.concat(list_of_data, ignore_index=True)


In [19]:
# Export excelfile with current date

OUTPUT_PATH = os.path.join(os.getcwd(), "output_data")

current_date = datetime.now().strftime("%Y-%m-%d")
output_filename = f"{current_date}_growth_data.xlsx"
output_path = os.path.join(OUTPUT_PATH, output_filename)
all_data.to_excel(output_path, index=False)